<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=dfaa96da60b675aa210b0d2ff9d0b58100f825d0cf47483f6ec68b2752b0260b
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-09 10:06:10
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.43 C
-------------------
Today PnL: -14.64 K (-0.1%)
Current PnL: -26.51 L (-16.91%)
CY Booked + Current PnL: -12.22 L (-7.8%)
-------------------
Total profit:  1.86 L
Total loss:  -28.37 L
-------------------
Total Booked + Current PnL: 14.76 L (11.37%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.02%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 91.97 L (64.51%)
Deployed:  1.30 C
Current:  1.43 C
CAGR/XIRR %: 5.11%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.20,14.99,5.57,21.40,17000.0,39780.0,305206.0,6.71,62.0,X-LC,5.72,14.0,2.34,2.14,22.26,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,-0.27,-21.62,27.69,0.09,21873.0,-21784.0,78993.0,83.35,34.0,M-SC,3.32,253.0,-1.00,0.55,3.78,OX40N,NTT,DURABLES
17,COALINDIA,484.83,0.08,4.66,13.92,19.23,21920.0,7012.0,157472.0,22.81,74.0,L-LC,7.14,182.0,0.32,1.10,26.93,XY25,ATH,MINING
39,INDIGOPNTS,1408.00,-0.25,-13.48,15.61,0.02,23574.0,-23539.0,151020.0,130.47,57.0,M-SC,8.09,240.0,-1.00,1.06,31.36,OX40N,NTT,PAINTS
49,MASFIN,398.61,-1.62,-3.23,26.12,22.05,24766.0,-3165.0,94815.0,-16.44,48.0,H-SC,6.53,168.0,-0.13,0.67,37.82,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,2.40,-17.20,51.58,25.51,79480.0,-32005.0,154090.0,-5.41,46.0,H-LC,1.72,50.0,-0.40,1.08,13.55,AR,ATH,ELECTRICAL
31,HAVELLS,2069.16,1.91,-3.19,36.85,32.48,117006.0,-10471.0,317520.0,-8.10,69.0,X-MC,5.15,69.0,-0.09,2.23,8.78,X40,ATH,ELECTRICAL
1,ABB,7934.00,1.73,-6.28,55.32,45.57,135636.0,-16435.0,245184.0,-39.74,46.0,H-MC,4.68,121.0,-0.12,1.72,4.33,AR,NTT,ELECTRICAL
5,ASIANPAINT,3460.25,1.49,-1.15,22.36,20.95,55646.0,-2904.0,248864.0,-2.18,55.0,X-LC,12.25,36.0,-0.05,1.75,34.07,X40,ATH,PAINTS
44,JCHAC,2282.00,1.36,-33.96,51.40,-0.01,43386.0,-43396.0,84409.0,17182.61,52.0,M-SC,13.73,235.0,-1.00,0.59,8.03,OX40N,BTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.00,-3.03,1.86,45.91,48.63,93506.0,3718.0,203672.0,-32.21,63.0,H-SC,13.59,137.0,0.04,1.43,13.70,XR,NTT,MISC
76,TRIDENT,48.00,-2.91,-29.53,82.37,28.51,53549.0,-27244.0,65010.0,-16.62,40.0,M-SC,7.40,226.0,-0.51,0.46,11.24,XR,NTT,TEXTILES
4,ALKYLAMINE,4546.37,-2.79,-33.45,190.93,93.63,128297.0,-33767.0,67196.0,-28.73,42.0,H-SC,16.63,143.0,-0.26,0.47,1.59,SR,ATH,CHEMICALS
61,ROUTE,2227.21,-2.40,-48.60,228.91,69.05,156556.0,-64674.0,68392.0,-58.36,43.0,H-SC,18.87,140.0,-0.41,0.48,5.48,SR,ATH,IT
56,RAJOOENG,143.10,-2.00,-31.27,103.12,39.61,72648.0,-32050.0,70450.0,-53.53,53.0,H-SC,30.16,136.0,-0.44,0.49,14.03,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,-0.36,0.18,20.52,20.74,41764.0,358.0,203528.0,31.15,57.0,M-MC,5.38,189.0,0.01,1.43,10.81,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.35,-1.40,110.87,107.91,161393.0,-2074.0,145570.0,-19.74,54.0,M-SC,10.10,220.0,-0.01,1.02,8.54,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.25,-13.48,15.61,0.02,23574.0,-23539.0,151020.0,130.47,57.0,M-SC,8.09,240.0,-1.00,1.06,31.36,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,0.60,-17.12,59.31,32.03,64802.0,-22575.0,109260.0,-10.33,66.0,H-SC,6.87,157.0,-0.35,0.77,22.38,OX40N,NTT,IT
77,TTKPRESTIG,770.00,-0.27,-21.62,27.69,0.09,21873.0,-21784.0,78993.0,83.35,34.0,M-SC,3.32,253.0,-1.00,0.55,3.78,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.14,-21.81,45.13,13.47,95153.0,-58824.0,210843.0,-67.92,49.0,H-SC,4.57,159.0,-0.62,1.48,7.21,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,-0.36,0.18,20.52,20.74,41764.0,358.0,203528.0,31.15,57.0,M-MC,5.38,189.0,0.01,1.43,10.81,XY25,NTT,FINANCE
17,COALINDIA,484.83,0.08,4.66,13.92,19.23,21920.0,7012.0,157472.0,22.81,74.0,L-LC,7.14,182.0,0.32,1.10,26.93,XY25,ATH,MINING
84,WIPRO,420.00,0.08,7.62,60.06,72.27,109530.0,12920.0,182368.0,-7.44,50.0,M-LC,6.17,101.0,0.12,1.28,14.27,XR,NTT,IT
37,IEX,219.00,-3.03,1.86,45.91,48.63,93506.0,3718.0,203672.0,-32.21,63.0,H-SC,13.59,137.0,0.04,1.43,13.70,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.08,7.62,60.06,72.27,109530.0,12920.0,182368.0,-7.44,50.0,M-LC,6.17,101.0,0.12,1.28,14.27,XR,NTT,IT
37,IEX,219.00,-3.03,1.86,45.91,48.63,93506.0,3718.0,203672.0,-32.21,63.0,H-SC,13.59,137.0,0.04,1.43,13.70,XR,NTT,MISC
25,FINCABLES,1641.55,0.35,-1.40,110.87,107.91,161393.0,-2074.0,145570.0,-19.74,54.0,M-SC,10.10,220.0,-0.01,1.02,8.54,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.05,-7.86,124.36,106.72,141326.0,-9693.0,113643.0,-54.44,34.0,H-SC,7.40,138.0,-0.07,0.80,13.68,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.69,-4.96,42.70,35.63,85053.0,-10387.0,199187.0,-18.51,38.0,H-MC,3.16,119.0,-0.12,1.40,10.89,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.29,-16.69,32.98,10.78,66026.0,-40117.0,200201.0,-50.55,13.0,X-LC,6.27,1.0,-0.61,1.40,0.0,X40,NTT,FMCG
7,AWL,485.00,-0.51,-26.53,113.95,57.19,273037.0,-86516.0,239611.0,-65.29,26.0,X-MC,9.20,58.0,-0.32,1.68,0.0,XY24,NTT,FMCG
52,PAGEIND,51605.08,-0.13,-12.12,48.63,30.62,101306.0,-28734.0,208320.0,-37.58,28.0,X-MC,16.38,57.0,-0.28,1.46,0.0,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-0.72,-40.62,127.70,35.20,97565.0,-52268.0,76402.0,0.22,30.0,X-SC,17.36,93.0,-0.54,0.54,0.0,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.81,-17.56,59.22,31.26,112660.0,-40528.0,190240.0,-61.53,33.0,X-MC,18.85,56.0,-0.36,1.33,1.4,X40N,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,0.28,-0.54,21.42,20.77,49885.0,-1256.0,232888.0,-9.12,35.0,X-MC,0.93,70.0,-0.03,1.63,6.34,X40N,NTT,BREWERIES
65,SIEMENS,4671.50,2.40,-17.20,51.58,25.51,79480.0,-32005.0,154090.0,-5.41,46.0,H-LC,1.72,50.0,-0.40,1.08,13.55,AR,ATH,ELECTRICAL
45,JIOFIN,387.00,-0.12,-5.69,31.99,24.48,80663.0,-15213.0,252152.0,-7.26,43.0,X-LC,1.73,37.0,-0.19,1.77,46.15,XY24,BTT,FINANCE
20,DABUR,735.00,0.51,2.08,40.78,43.70,103262.0,5150.0,253218.0,-6.31,66.0,X-MC,2.24,73.0,0.05,1.78,17.44,XY24,BTT,FMCG
81,VBL,671.64,0.22,1.40,33.79,35.67,108221.0,4434.0,320276.0,-10.61,63.0,X-LC,2.28,5.0,0.04,2.25,15.45,X40N,ATH,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.20,-27.72,127.09,64.15,218595.0,-65951.0,172000.0,-57.62,36.0,X-SC,6.24,84.0,-0.30,1.21,0.0,XY24,BTT,CEMENT
7,AWL,485.00,-0.51,-26.53,113.95,57.19,273037.0,-86516.0,239611.0,-65.29,26.0,X-MC,9.20,58.0,-0.32,1.68,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.72,-40.62,127.70,35.20,97565.0,-52268.0,76402.0,0.22,30.0,X-SC,17.36,93.0,-0.54,0.54,0.0,X40,NTT,FOOTWEAR
43,ITC,452.00,-0.29,-16.69,32.98,10.78,66026.0,-40117.0,200201.0,-50.55,13.0,X-LC,6.27,1.0,-0.61,1.40,0.0,X40,NTT,FMCG
52,PAGEIND,51605.08,-0.13,-12.12,48.63,30.62,101306.0,-28734.0,208320.0,-37.58,28.0,X-MC,16.38,57.0,-0.28,1.46,0.0,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-1.83,-27.87,98.05,42.84,45972.0,-18120.0,46886.0,-52.62,50.0,X-SC,27.03,83.0,-0.39,0.33,6.07,XY24,NTT,MISC
58,RELAXO,1176.00,0.10,-48.16,193.78,52.31,145831.0,-69904.0,75256.0,-44.83,42.0,X-SC,7.81,92.0,-0.48,0.53,0.73,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.72,-40.62,127.70,35.20,97565.0,-52268.0,76402.0,0.22,30.0,X-SC,17.36,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,0.97,1.25,22.55,24.08,28985.0,1586.0,128536.0,-6.93,53.0,X-SC,6.16,91.0,0.05,0.90,21.11,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.13,-17.27,73.24,43.32,98684.0,-28132.0,134740.0,-28.92,46.0,X-SC,7.84,90.0,-0.29,0.95,3.46,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.29,-16.69,32.98,10.78,66026.0,-40117.0,200201.0,-50.55,13.0,X-LC,6.27,1.0,-0.61,1.40,0.00,X40,NTT,FMCG
75,TMPV,600.00,-0.47,-26.68,67.71,22.98,112153.0,-60260.0,165638.0,-24.30,45.0,X-LC,4.09,2.0,-0.54,1.16,4.18,XY24,NTT,AUTO
73,TCS,4389.96,0.36,-11.59,36.53,20.71,122157.0,-43835.0,334402.0,-23.95,48.0,X-LC,2.49,3.0,-0.36,2.35,11.74,X40,ATH,IT
81,VBL,671.64,0.22,1.40,33.79,35.67,108221.0,4434.0,320276.0,-10.61,63.0,X-LC,2.28,5.0,0.04,2.25,15.45,X40N,ATH,FMCG
41,INFY,1972.00,0.21,10.06,21.98,34.24,75690.0,31469.0,344357.0,-14.21,51.0,X-LC,7.68,6.0,0.42,2.42,19.17,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.45,-34.48,102.83,32.89,52017.0,-26622.0,50585.0,-724.57,59.0,L-MC,7.75,259.0,-0.51,0.35,39.31,XR,NTT,BANKS
13,BSOFT,831.70,-0.44,-22.09,93.53,50.79,102496.0,-31067.0,109586.0,0.12,51.0,H-SC,10.71,171.0,-0.30,0.77,27.59,XR,ATH,IT
69,SURYODAY,216.00,0.80,-16.75,47.79,23.03,71242.0,-29998.0,149073.0,62.28,61.0,H-SC,4.15,165.0,-0.42,1.05,47.91,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.25,-13.48,15.61,0.02,23574.0,-23539.0,151020.0,130.47,57.0,M-SC,8.09,240.0,-1.00,1.06,31.36,OX40N,NTT,PAINTS
17,COALINDIA,484.83,0.08,4.66,13.92,19.23,21920.0,7012.0,157472.0,22.81,74.0,L-LC,7.14,182.0,0.32,1.10,26.93,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,1.04,-28.11,57.04,12.90,89600.0,-61414.0,157082.0,131.97,65.0,H-SC,10.19,173.0,-0.69,1.10,30.61,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,0.60,-17.12,59.31,32.03,64802.0,-22575.0,109260.0,-10.33,66.0,H-SC,6.87,157.0,-0.35,0.77,22.38,OX40N,NTT,IT
17,COALINDIA,484.83,0.08,4.66,13.92,19.23,21920.0,7012.0,157472.0,22.81,74.0,L-LC,7.14,182.0,0.32,1.10,26.93,XY25,ATH,MINING
37,IEX,219.00,-3.03,1.86,45.91,48.63,93506.0,3718.0,203672.0,-32.21,63.0,H-SC,13.59,137.0,0.04,1.43,13.70,XR,NTT,MISC
82,VOLTAS,1856.00,0.54,15.55,25.66,45.20,56850.0,29808.0,221550.0,4.31,64.0,X-MC,4.94,80.0,0.52,1.55,23.40,XY25,NTT,AC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.34
1,25,44.37
2,50,76.46


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.85
MC    30.72
LC    26.46
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.36
X40      23.92
X40N     12.82
XY25     10.67
XR        9.06
AR        8.56
OX40N     7.69
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.33
X-LC    23.00
H-SC    22.92
M-SC    11.29
X-SC     7.85
H-MC     4.61
M-MC     1.43
M-LC     1.28
L-LC     1.10
H-LC     1.08
L-SC     0.79
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.06,-8.32,43.85
IT,12.83,-16.93,75.91
FINANCE,11.07,-13.05,60.49
MISC,7.17,-27.18,79.20
ELECTRICAL,5.98,-10.22,50.59
PAINTS,5.76,-12.61,29.33
INSURANCE,4.73,-0.78,35.26
PHARMA,3.92,-5.25,38.38
AUTO,2.78,-32.03,77.79


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3213620.0,21
AR,1351942.0,10
XR,1201694.0,12
X40,1113160.0,15
X40N,915387.0,9
OX40N,696508.0,10
XY25,419757.0,7
SR,284853.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3528453.0,24
X-MC,1564568.0,16
M-SC,1449441.0,15
X-LC,945913.0,13
X-SC,802969.0,8
H-MC,421985.0,3
L-SC,178881.0,2
M-LC,109530.0,1
H-LC,79480.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1247856.0      6
           AR         921597.0      5
M-SC       XY24       835638.0      6
H-SC       XR         770327.0      7
X-MC       X40        500899.0      7
           XY24       442462.0      3
X-LC       X40        415859.0      6
X-MC       X40N       398555.0      4
X-SC       X40N       313015.0      3
H-SC       OX40N      303820.0      4
M-SC       OX40N      300727.0      5
X-SC       XY24       293552.0      3
H-SC       SR         284853.0      2
X-MC       XY25       222652.0      2
H-MC       AR         220689.0      2
X-LC       X40N       203817.0      2
H-MC       XY24       201296.0      1
X-SC       X40        196402.0      2
X-LC       XY24       192816.0      2
M-SC       XR         182900.0      2
X-LC       XY25       133421.0      3
M-SC       AR         130176.0      2
M-LC       XR         109530.0      1
L-SC       OX40N       91961.0      1
           XR          86920.0      1
H-LC       AR          79480.0      1
L-MC       XR          52017.0      1
M-MC       XY25        41764.0      1
L-LC       XY25        21920.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
